# Building Your First AI Agent

This notebook a step by step guide on how to build your own Multi-agent AI system to perform web search and collect top 10 research papers published in ArXiv for a particular topic based on date.


Refer to the below substack for better understanding of each module. https://www.intoai.pub/p/building-your-first-ai-agent

In [1]:
# Set up environment variables
import os
# Test the model
from openai import OpenAI
from dotenv import load_dotenv
import http

from crewai_tools import (  # pyright: ignore[reportMissingImports]
    DirectoryReadTool,
    FileReadTool,
    SerperDevTool,
    WebsiteSearchTool
)

from typing import Type, List
from pydantic import BaseModel, Field 
from crewai.tools import BaseTool   # pyright: ignore[reportMissingImports]
from crewai import LLM, Agent, Task, Crew  # pyright: ignore[reportMissingImports]

import arxiv  # pyright: ignore[reportMissingImports]
import time
import datetime

load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
serper_api_key = os.getenv("SERPER_API_KEY")

In [2]:
# Test if the keys are working

client = OpenAI(
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

response = client.responses.create(
    input="What are transformers in Deep Learning. Explain in 2 lines only",
    model="llama-3.1-8b-instant",
)
print(response.output_text)


Transformers are a type of neural network architecture used in deep learning for natural language processing (NLP) and other sequential data tasks, utilizing self-attention mechanisms to weigh input elements based on relevance. They are particularly effective in understanding context and long-range dependencies in data.


In [3]:
conn = http.client.HTTPSConnection("google.serper.dev")
payload = ''
headers = {}
conn.request("GET", "/search?q=cat&apiKey=616509eec3f7c2d80906ae20249417939e52970a", payload, headers)
res = conn.getresponse()
data = res.read()
print(data.decode("utf-8"))

{"searchParameters":{"q":"cat","type":"search","engine":"google"},"organic":[{"title":"Cat","link":"https://en.wikipedia.org/wiki/Cat","snippet":"The cat (Felis catus), also called domestic cat and house cat, is a small carnivorous mammal. It is an obligate carnivore, requiring a predominantly meat-based ...","position":1},{"title":"Caterpillar: Products & Services – North America | Cat","link":"https://www.cat.com/en_US.html","snippet":"Cat® equipment and services are backed by the largest, most experienced dealer network in the world. Get the job done with our products, technology and ...","position":2},{"title":"ALL THE CATS, EXPLAINED","link":"https://www.youtube.com/watch?v=UWagM1FBB4c","snippet":"Wild felines come in all shapes and sizes and live all over the world plus there are more than half a billion domestic cats.","date":"2 months ago","position":3},{"title":"Domestic cat","link":"https://www.nationalgeographic.com/animals/mammals/facts/domestic-cat","snippet":"As mostly no

In [4]:
# Instantiate tools
docs_tool = DirectoryReadTool(directory='./your-directory') # Tool for reading documents from a specified directory
file_tool = FileReadTool() # Tool for reading individual files
search_tool = SerperDevTool() # Tool for performing web searches using the Serper API
web_rag_tool = WebsiteSearchTool() # Tool for searching and extracting information from websites

In [5]:
class FetchArxivPapersInput(BaseModel):
    """Input schema for FetchArxivPapersTool."""
    target_date: datetime.date = Field(..., description="Target date to fetch papers for.")

class FetchArxivPapersTool(BaseTool):
    name: str = "fetch_arxiv_papers"
    description: str = "Fetches all ArXiv papers from selected categories submitted on the target date."
    args_schema: Type[BaseModel] = FetchArxivPapersInput

    def _run(self, target_date) -> List[dict]:
        # Accept string dates from the LLM (e.g. "2025-03-12")
        if isinstance(target_date, str):
            target_date = datetime.datetime.strptime(target_date, "%Y-%m-%d").date()
        # List of AI-related categories. 
        # You can also include ["cs.AI", "cs.LG", "cs.CV", "cs.MA", "cs.RO"]
        AI_CATEGORIES = ["cs.CL"]

        # Define the date range for the target date
        start_date = target_date.strftime('%Y%m%d%H%M')
        end_date = (target_date + datetime.timedelta(days=1)).strftime('%Y%m%d%H%M')

        # Initialize the ArXiv client
        client = arxiv.Client(
            page_size=100,  # Fetch 100 results per page
            delay_seconds=3  # Delay between requests to respect rate limits
        )

        all_papers = []

        for category in AI_CATEGORIES:
            print(f"Fetching papers for category: {category}")

            search_query = f"cat:{category} AND submittedDate:[{start_date} TO {end_date}]"

            search = arxiv.Search(
                query=search_query,
                sort_by=arxiv.SortCriterion.SubmittedDate,
                max_results=None  # Fetch all results
            )

            # Collect results for the category
            category_papers = []
            for result in client.results(search):
                category_papers.append({
                    'title': result.title,
                    'authors': [author.name for author in result.authors],
                    'summary': result.summary,
                    'published': result.published,
                    'url': result.entry_id
                })

                # Delay between requests to respect rate limits
                time.sleep(3)

            print(f"Fetched {len(category_papers)} papers from {category}")
            all_papers.extend(category_papers)

        return all_papers

In [6]:
arxiv_search_tool = FetchArxivPapersTool()

In [7]:
llm = LLM(
    model="groq/llama-3.3-70b-versatile",
    temperature=0.5,
    api_key=groq_api_key,
    base_url="https://api.groq.com/openai/v1",
)

In [8]:
researcher = Agent(
    role = "Senior Researcher",
    goal = "Find the top 10 papers from the search results from ArXiv on {date}."
            "Rank them appropirately.",
    backstory = "You are a senior researcher with a deep understanding of all topics in AI and AI research."
                "You are able to identify the best research papers based on the title and abstract.",
    verbose = True,
    llm = llm,
    tools = [arxiv_search_tool],
)

In [9]:
# Agent 2: Frontend Engineer

frontend_engineer = Agent(
    role = "Senior Frontend & AI Engineer",
    goal = "Compile the results into a HTML file.",
    backstory = "You are a competent frontend engineer writing HTML and CSS with decades of experience."
                "You have also been working with AI for decades and understand it well.",
    verbose = True,
    llm = llm
)

In [10]:
# Task for ArXiv Researcher

research_task = Task(
    description = (" Find the top 10 research papers from the search results from ArXiv on {date}."),
    expected_output = (
        "A list of top 10 research papers with the following information in the following format:"
        "- Title"
        "- Authors"
        "- Abstract"
        "- Link to the paper"
    ),
    agent = researcher,
    human_input = True,
)

In [11]:
# Task for Frontend Engineer

reporting_task = Task(
    description = ("Compile the results into a detailed report in a HTML file."),
    expected_output = (
        "An HTML file with the results in the following format:"
        "Top 10 AI Research Papers published on {date}"
        "- Title (which on clicking opens the paper in a new tab)"
        "- Authors"
        "- Short summary of the abstract (2-4 sentences)"
    ),
    agent = frontend_engineer,
    context = [research_task],
    output_file = "ai_research_report.html",
    human_input = True,
)

In [12]:
arxiv_research_crew = Crew(
    agents = [researcher, frontend_engineer],
    tasks = [research_task, reporting_task],
    verbose = True,
)

In [13]:
crew_inputs = {
    "date" : "2025-03-12"
}

result = arxiv_research_crew.kickoff(inputs = crew_inputs)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e3c65ff4-354c-4924-b3f0-8bf75969aa0f                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:  Find the top 10 research papers from the search results from ArXiv on 2025-03-12.                       │
│  ID: da589c29-1ec5-4c59-adc2-97d1a4bc3adf                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Researcher                                                                                       │
│                                                                                                                 │
│  Task:  Find the top 10 research papers from the search results from ArXiv on 2025-03-12.                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[CrewAIEventsBus] Warning: Event pairing mismatch. 'tool_usage_finished' closed 'llm_call_started' (expected 
'tool_usage_started')

╭──────────────────────────────────────── 🔧 Tool Execution Started (#1) ─────────────────────────────────────────╮
│                                                                                                                 │
│  Tool: fetch_arxiv_papers                                                                                       │
│  Args: {'target_date': '2025-03-12'}                                                                            │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Frontend & AI Engineer                                                                           │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```html                                                                                                        │
│  <!DOCTYPE html>                                                                                                │
│  <html>                                                                                                         │
│  <head>                                                                                                         │
│          <title>Top 10 AI Research Papers</title>                                                               │
│          <style>                                                                                                │
│                  body {                                                                                         │
│                          font-family: Arial, sans-serif;                                                        │
│                  }                                                                                              │
│                  .paper {                                                                                       │
│                          margin-bottom: 20px;                                                                   │
│                          padding: 20px;                                                                         │
│                          border: 1px solid #ddd;                                                                │
│                          border-radius: 10px;                                                                   │
│                          box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);                                               │
│                  }                                                                                              │
│                  .paper-title {                                                                                 │
│                          font-size: 18px;                                                                       │
│                          font-weight: bold;                                                                     │
│                          margin-bottom: 10px;                                                                   │
│                  }                                                                                              │
│                  .paper-authors {                                                                               │
│                          font-size: 16px;                                                                       │
│                          color: #666;                                                                           │
│                          margin-bottom: 10px;                                                                   │
│                  }                                                                                              │
│                  .paper-summary {                                                                               │
│                          font-size: 14px;                                                                       │
│                          color: #888;                                                                           │
│                  }                                     

╭────────────────────────────────────────── 💬 Human Feedback Required ───────────────────────────────────────────╮
│                                                                                                                 │
│  Provide feedback on the Final Result above.                                                                    │
│                                                                                                                 │
│  • If you are happy with the result, simply hit Enter without typing anything.                                  │
│  • Otherwise, provide specific improvement requests.                                                            │
│  • You can provide multiple rounds of feedback until satisfied.                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────── 📋 Task Completion ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name:                                                                                                          │
│  Compile the results into a detailed report in a HTML file.                                                     │
│  Agent:                                                                                                         │
│  Senior Frontend & AI Engineer                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  e3c65ff4-354c-4924-b3f0-8bf75969aa0f                                                                           │
│  Final Output: ```html                                                                                          │
│  <!DOCTYPE html>                                                                                                │
│  <html>                                                                                                         │
│  <head>                                                                                                         │
│          <title>Top 10 AI Research Papers</title>                                                               │
│          <style>                                                                                                │
│                  body {                                                                                         │
│                          font-family: Arial, sans-serif;                                                        │
│                  }                                                                                              │
│                  .paper {                                                                                       │
│                          margin-bottom: 20px;                                                                   │
│                          padding: 20px;                                                                         │
│                          border: 1px solid #ddd;                                                                │
│                          border-radius: 10px;                                                                   │
│                          box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);                                               │
│                  }                                                                                              │
│                  .paper-title {                                                                                 │
│                          font-size: 18px;                                                                       │
│                          font-weight: bold;                                                                     │
│                          margin-bottom: 10px;                                                                   │
│                  }                                                                                              │
│                  .paper-authors {                                                                               │
│                          font-size: 16px;                                                                       │
│                          color: #666;                                                                           │
│                          margin-bottom: 10px;                                                                   │
│                  }                                                                                              │
│                  .paper-summary {                                                                               │
│                          font-size: 14px;             

╭──────────────────────────────────────────────── Tracing Status ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Info: Tracing is disabled.                                                                                     │
│                                                                                                                 │
│  To enable tracing, do any one of these:                                                                        │
│  • Set tracing=True in your Crew/Flow code                                                                      │
│  • Set CREWAI_TRACING_ENABLED=true in your project's .env file                                                  │
│  • Run: crewai traces enable                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯